In [29]:
#first lets look at the data and analyse the things that can be cleaned 


In [30]:
from datetime import datetime

In [31]:
import pandas as pd  #This is a really popular python library used for importing and analysing the data

df = pd.read_csv("car.csv")  #reads from the file car.csv
df.head() #displays the first 5 elements from the top in a proper format


,name,year,selling_price,km_driven,fuel,seller_type,transmission,owner,mileage(km/ltr/kg),engine,max_power,seats
0,Maruti Swift Dzire VDI,2014,450000,145500,Diesel,Individual,Manual,First Owner,23.40,1248.0,74,5.0
1,Skoda Rapid 1.5 TDI Ambition,2014,370000,120000,Diesel,Individual,Manual,Second Owner,21.14,1498.0,103.52,5.0
2,Honda City 2017-2020 EXi,2006,158000,140000,Petrol,Individual,Manual,Third Owner,17.70,1497.0,78,5.0
3,Hyundai i20 Sportz Diesel,2010,225000,127000,Diesel,Individual,Manual,First Owner,23.00,1396.0,90,5.0
4,Maruti Swift VXI BSIII,2007,130000,120000,Petrol,Individual,Manual,First Owner,16.10,1298.0,88.2,5.0


In [32]:
df.rename(columns={'mileage(km/ltr/kg)': 'mileage'}, inplace=True) #renaming for better readability


In [33]:
#Since we are not able to get enough information just from these 5 datas, we will look at some more using
df.head(50)


#Just by looking at it, We can see a few information. Fuel type can be Disel, Petrol, LPG
#seller_type can be either individual or Dealer
#or to make life simpler, we can just the library functions of pandas

print("Fuel Types:", df['fuel'].unique()) 
print("Owner Types:", df['owner'].unique())
print("Transmission Types:", df['transmission'].unique()) 
print("Seller Types", df["seller_type"].unique())


#These basically scan through the entire dataset, and get all the unique values in the sepcific column that we have mentioned


Fuel Types: ['Diesel' 'Petrol' 'LPG' 'CNG']
Owner Types: ['First Owner' 'Second Owner' 'Third Owner' 'Fourth & Above Owner'
 'Test Drive Car']
Transmission Types: ['Manual' 'Automatic']
Seller Types ['Individual' 'Dealer' 'Trustmark Dealer']


In [34]:
#Before cleaning data, lets use some LLM's (Chatgpt and Claude.ai) to understand each columns better

#The first question I asked is, "which of the data columns will have a significant impact on the selling_price?" 
#year: Newer cars usually sell for more. Older ones depreciate. A derived feature like car_age = current_year - year will be even more useful.
#years_drived: More kilometers driven = more wear and tear = usually lower price.
#owner: A first-owner car is usually priced higher than a third- or fourth-owner car.

#These three are the major ones affecting the price in a significant manner


In [35]:
#now to clean the data, first we will handle the normal procedures

#Removing duplicates, removing empty values, Cinverting datatypes

In [36]:
#first we convert the year to car age for easier predictions
df['car_age'] = datetime.now().year - df['year']
df['car_age']

0       11
1       11
2       19
3       15
4       18
        ..
8123    12
8124    18
8125    16
8126    12
8127    12
Name: car_age, Length: 8128, dtype: int64

In [37]:
#removing duplicates and empty/null values


print("Duplicate rows:", df.duplicated().sum()) #checking the number of duplicate rows in the dataset (duplicate rows just add pressure while training, so we remove it)
df.drop_duplicates(inplace=True) #removing/dropping duplicate rows from the dataset





Duplicate rows: 1202


In [38]:
print(df.isnull().sum()) 
df.dropna(inplace=True)

name               0
year               0
selling_price      0
km_driven          0
fuel               0
seller_type        0
transmission       0
owner              0
mileage          208
engine           208
max_power        205
seats            208
car_age            0
dtype: int64


In [39]:
#checking the datatype of each column to see if any columns needs changes
df.dtypes

#since max_power is in object, we will convert it into float datatype
df['max_power'] = df['max_power'].astype(str).str.extract(r'(\d+\.\d+|\d+)').astype(float) #uses regex to pull out only the numeric part and converts it completely to float dtype


In [40]:
df.dtypes


name              object
year               int64
selling_price      int64
km_driven          int64
fuel              object
seller_type       object
transmission      object
owner             object
mileage          float64
engine           float64
max_power        float64
seats            float64
car_age            int64
dtype: object

In [41]:
#next thing that I analysed is, we can convert "owner" to numerical format for proper predictions
#lets first check the unique values
print("Owner Types:", df['owner'].unique())


Owner Types: ['First Owner' 'Second Owner' 'Third Owner' 'Fourth & Above Owner'
 'Test Drive Car']


In [42]:
#replacing the values accordingly
owner_mapping = {
    'First Owner': 0,
    'Second Owner': 1,
    'Third Owner': 2,
    'Fourth & Above Owner': 3,
    'Test Drive Car': -1 
}

df['owner'] = df['owner'].map(owner_mapping)


In [43]:
print("Owner Types:", df['owner'].unique())

Owner Types: [ 0  1  2  3 -1]
